In [21]:
# Import NUPACK Python module
from nupack import *

rna06_nupack4_parameter=["rna06", "Based on [Mathews99] and [Lu06] with additional parameters [Xia98,Zuker03] including coaxial stacking [Mathews99,Turner10] and dangle stacking [Serra95,Zuker03,Turner10] in 1M Na+."]
rna95_nupack4_parameters=["rna95", "Based on [Serra95] with additional parameters [Zuker03] including coaxial stacking [Mathews99,Turner10] and dangle stacking [Serra95,Zuker03,Turner10] in 1M Na+."]
rna99_nupack3_parameters=["rna99-nupack3","Parameters from [Mathews99] with terminal mismatch free energies in exterior loops and multiloops replaced by two dangle stacking free energies. Parameters are provided only for 37 ∘C."]
rna95_nupack3_parameters=["rna95-nupack3", "Same as rna95 except that terminal mismatch free energies in exterior loops and multiloops are replaced by two dangle stacking free energies."]

rna_parameterSet=[rna06_nupack4_parameter, rna95_nupack4_parameters, rna99_nupack3_parameters, rna95_nupack3_parameters]

print("Enter single strand RNA sequence")
sequence = input()

print("Enter the temperature to simulate at")
temperature = input()

print("Select the energy parameter set you wish to use. enter the number that corresponds")
for parameterSet_index in range(len(rna_parameterSet)):
    parameterSet="#={0}, name={1}, description={2}".format(parameterSet_index, rna_parameterSet[parameterSet_index][0], rna_parameterSet[parameterSet_index][1])
    print(parameterSet)

paramNum=int(input())

                                                           
rna_model= rna_parameterSet[paramNum][0]                                                         

rna_model='rna99-nupack3'    
# Define physical model 

def energy_model(rna_model, temperature):
    model=Model(material=rna_model, celsius=int(temperature))
    return model


my_model = energy_model(rna_model, temperature)


# Partition function and complex free energy
partionFunction, freeEnergy_complex = pfunc(strands=sequence, model=my_model)

# Equilibrium pair probability matrix
probabilityMatrix_Raw = pairs(strands=sequence, model=my_model)


def get_subOpt_kcal_delta_elements(sequence_string, energymodel, energy_delta_MFE):
    #run through subopt 
    ensemble_kcal_group= subopt(strands=sequence_string, model=energymodel, energy_gap=energy_delta_MFE)
    
    #get all the data out of it
    kcal_group_elements_list=[]
    for i,kcal_group_elementInfo in enumerate(ensemble_kcal_group):        
        #get all the structures and energis pulled and prepped for proccessing and add them tot eh dict and the list
        element_info_Dict = {}        
        element_structure = str(kcal_group_elementInfo.structure)
        element_info_Dict["structure"] = element_structure
        element_freeEnergy = kcal_group_elementInfo.energy
        element_info_Dict["freeEnergy"] = element_freeEnergy
        element_stackEnergy = kcal_group_elementInfo.stack_energy
        element_info_Dict["stackEnergy"] = element_stackEnergy
        kcal_group_elements_list.append(element_info_Dict)

    return kcal_group_elements_list



def get_EV(ensemble_kcal_group_elements, sequence):
    #need to do each char abd then structure
    #walk through each nucleotide but first prep containers grab what is needed
    
    #setup constants
    nuc_count = len(sequence)
    structure_element_count = len(ensemble_kcal_group_elements)
    
    
    ensembleVariation_score_temp = 0
    nucleotide_position_variation_basescores=[0]*nucleotide_count
    nucleotide_position_variation_subscores=[0]*nucleotide_count
    
    #we have all the elements and all the stuff for the group
    for nucIndex in range(nuc_count):
        for structureIndex in range(structure_element_count):
            mfe_nuc=ensemble_kcal_group_elements[0]["structure"][nucIndex]
            delta_nuc=ensemble_kcal_group_elements[structureIndex]["structure"][nucIndex]
            if mfe_nuc != delta_nuc:
                #then add a point since there is variation if nuc dotbracket not the same
                #this is a subscore for that nuc position only across the ensemble
                nucleotide_position_variation_basescores[nucIndex]=nucleotide_position_variation_basescores[nucIndex]+1
        #when done with all teh structures then do the nucScore wityh calculations
        nucleotide_position_variation_subscores[nucIndex]=nucleotide_position_variation_basescores[nucIndex] / structure_element_count
    #when that is all done the calculate final score
    total_subscore=sum(nucleotide_position_variation_subscores)
    
    #now do final calculations
    # original classic EV and normalized classic EV
    nucleotide_ensemble_variance_classic=total_subscore/nuc_count    
    fail_threshold=0.2
    nucleotide_ensemble_variance_normalized=(nucleotide_ensemble_variance_classic/fail_threshold)*100
    
    
    #new method that uses structure distance in nupack for points... distance is points
    
    energydelta_individualVariationScore_list=[]
    for element in ensemble_kcal_group_elements:
        element_structure = element["structure"]
        mfe_structure = ensemble_kcal_group_elements[0]["structure"]
        element_structure_mfe_distance=struc_distance(element_structure, mfe_structure)
        individualVariationScore = element_structure_mfe_distance / structure_element_count
        energydelta_individualVariationScore_list.append(individualVariationScore)
    
    structure_ensemble_variance = sum(energydelta_individualVariationScore_list) * 100
    return nucleotide_ensemble_variance_classic, nucleotide_ensemble_variance_normalized, structure_ensemble_variance
    
mfe_info = get_subOpt_kcal_delta_elements(sequence, my_model,0)[0]
mfe_structure=mfe_info["structure"]
mfe_freeEnergy=mfe_info["freeEnergy"]
mfe_stackEnergy=mfe_info["stackEnergy"]

#now calulate ensemble variance
print("enter a list of kcal deltas to look at seperated by a single space")
energyDeltas_raw=input()
deltasToInspect_list=energyDeltas_raw.split(" ")
deltacount=len(deltasToInspect_list)

deltaMessage="Getting EV score for {0} deltas: {1}".format(deltacount, deltasToInspect_list)

#Get ensemble variance data
EV_Delta_dict={}
nucleotide_count=len(sequence)
for delta in deltasToInspect_list:
    delta = int(delta)
    delta_elements=get_subOpt_kcal_delta_elements(sequence, my_model,delta) 
    nucleotide_ensemble_variance_classic, nucleotide_ensemble_variance_normalized, structure_ensemble_variance=get_EV(delta_elements, sequence)
    EV_Delta_dict[delta]={}
    structures_count=len(delta_elements)
    
    EV_Delta_dict[delta]["count"]=structures_count 
    EV_Delta_dict[delta]["struct_elements"]=delta_elements    
    EV_Delta_dict[delta]["EV_old_raw"]=nucleotide_ensemble_variance_classic
    EV_Delta_dict[delta]["EV_old_norm"]=nucleotide_ensemble_variance_normalized   
    EV_Delta_dict[delta]["EV_new"]=structure_ensemble_variance
    #now we have everything in easy access for the group of ensemble

#Get ensemble defect
ensemble_defect=defect(strands=sequence, structure=mfe_structure, model=my_model)

#print out the data now
#print MFE styff
mfe_string = "MFE Secondary Structure = {0}\nMFE Free Energy={1}\nMFE Stack Energy={2}".format(mfe_structure, mfe_freeEnergy,mfe_stackEnergy)
print(mfe_string)

#print out NUPACK ED data
ensemble_defect_string = "Ensemble Defect (ED) = {0}".format(ensemble_defect)
print(ensemble_defect_string)

#print out JenniferPearl EV data and ensemble strucutre info stuctures
for deltaIndex in range(len(EV_Delta_dict)):
    #get eh list of keys
    thisDelta=list(EV_Delta_dict.keys())[deltaIndex]    
    kcalDelta=thisDelta
    EV_old=EV_Delta_dict[thisDelta]["EV_old_raw"]
    EV_old_norm=EV_Delta_dict[thisDelta]["EV_old_norm"]
    EV_new=EV_Delta_dict[thisDelta]["EV_new"]
    strutureCount=EV_Delta_dict[thisDelta]["count"]
    deltaMessage = "Kcal_Delta={0}, EV_old={1}, EV_old_norm={2}, EV_new={3}, structure_count={4}".format(kcalDelta,EV_old,EV_old_norm,EV_new,strutureCount)
    print(deltaMessage)
    print("Ensemble kcal delta secondary structures and energy's")
    index = 1
    for element in EV_Delta_dict[thisDelta]["struct_elements"]:
        element_structure=element["structure"]
        element_freeEnergy=element["freeEnergy"]
        element_stackEnergy=element["stackEnergy"]
        struct_message="{0}: {1} FreeEnergy: {2:+.2f}: Stack Energy: {3:+.2f}".format(index, element_structure ,element_freeEnergy, element_stackEnergy)
        print(struct_message)
        index += 1
    print("\n")
    
    

Enter single strand RNA sequence


 auauauagaaaauauaua


Enter the temperature to simulate at


 37


Select the energy parameter set you wish to use. enter the number that corresponds
#=0, name=rna06, description=Based on [Mathews99] and [Lu06] with additional parameters [Xia98,Zuker03] including coaxial stacking [Mathews99,Turner10] and dangle stacking [Serra95,Zuker03,Turner10] in 1M Na+.
#=1, name=rna95, description=Based on [Serra95] with additional parameters [Zuker03] including coaxial stacking [Mathews99,Turner10] and dangle stacking [Serra95,Zuker03,Turner10] in 1M Na+.
#=2, name=rna99-nupack3, description=Parameters from [Mathews99] with terminal mismatch free energies in exterior loops and multiloops replaced by two dangle stacking free energies. Parameters are provided only for 37 ∘C.
#=3, name=rna95-nupack3, description=Same as rna95 except that terminal mismatch free energies in exterior loops and multiloops are replaced by two dangle stacking free energies.


 2


enter a list of kcal deltas to look at seperated by a single space


 1 2


MFE Secondary Structure = .((((((.....))))))
MFE Free Energy=-1.695277214050293
MFE Stack Energy=-1.4000000953674316
Ensemble Defect (ED) = 0.2766794030152172
Kcal_Delta=1, EV_old=0.1111111111111111, EV_old_norm=55.55555555555555, EV_new=533.3333333333334, structure_count=3
Ensemble kcal delta secondary structures and energy's
1: .((((((.....)))))) FreeEnergy: -1.70: Stack Energy: -1.40
2: ((((((.....)))))). FreeEnergy: -1.17: Stack Energy: -1.00
3: ..(((((.....))))). FreeEnergy: -1.21: Stack Energy: -0.70


Kcal_Delta=2, EV_old=0.23809523809523803, EV_old_norm=119.04761904761901, EV_new=828.5714285714284, structure_count=7
Ensemble kcal delta secondary structures and energy's
1: .((((((.....)))))) FreeEnergy: -1.70: Stack Energy: -1.40
2: ((((((.....)))))). FreeEnergy: -1.17: Stack Energy: -1.00
3: ..(((((.....))))). FreeEnergy: -1.21: Stack Energy: -0.70
4: .(((((.....))))).. FreeEnergy: -0.59: Stack Energy: -0.10
5: .................. FreeEnergy: +0.00: Stack Energy: +0.00
6: ...(((